In [4]:
import datetime
import geopandas
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2023-05-05 14:47:42.389825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# TODO try only with best performing numeric columns for input into model first
# TODO select features for deep learning model after ranking features with random forest (feature importance)
# TODO lstm network
# TODO SMOTE algorithm, for upsampling havested fields; we only have ~300 :(

In [6]:
#local scripts
from scripts import veg_indices, utilities, plots
#from scripts.utilities import *

In [7]:
# cpied from learning_about-data.ipynb
BANDS_DICT = {   'B2': 'Blue',
            'B3': 'Green',
            'B4': 'Red',
            'B5': 'Red_Edge_1',
            'B6': 'Red_Edge_2',
            'B7': 'Red_Edge_3',
            'B8': 'NIR',
            'B8A': 'Red_Edge_4',
            'B11': 'SWIR_1',
            'B12': 'SWIR_2'}

BANDS = list(BANDS_DICT.keys())

In [8]:
# cpied from learning_about-data.ipynb
DF = geopandas.read_file('../data/merged_images.geojson')
DF.rename(columns = {'is_within_period':'har_evnt'}, inplace = True)
NUM_SAMPLES = len(np.unique(DF.image_idx)) - 1

In [9]:
DF

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,NDVI,finHarvDat,lat,lon,point_idx,start_date,end_date,har_evnt,image_idx,geometry
0,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,2022-08-11,50.655048,25.458684,p0,2022-01-08,2022-01-28,False,i0,POINT (25.45868 50.65505)
1,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.294633,36.289471,p1,2022-01-08,2022-01-28,False,i0,POINT (36.28947 50.29463)
2,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.860849,32.548484,p2,2022-01-08,2022-01-28,False,i0,POINT (32.54848 50.86085)
3,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.928689,31.637558,p3,2022-01-08,2022-01-28,False,i0,POINT (31.63756 50.92869)
4,0.1424,0.1083,0.030,0.0432,0.0631,0.07150,0.07760,0.08510,0.1066,0.1043,0.256335,2022-07-23,48.307964,37.348245,p4,2022-01-08,2022-01-28,False,i0,POINT (37.34824 48.30796)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,0.2557,0.1875,0.065,0.0740,0.0973,0.11695,0.14255,0.15525,0.1920,0.1847,0.327342,NaT,49.723156,36.817274,p553,2022-12-10,2022-12-30,False,i16,POINT (36.81727 49.72316)
9482,0.1616,0.1235,0.028,0.0442,0.0525,0.06390,0.08950,0.09310,0.1152,0.1073,0.373882,NaT,45.578351,28.700480,p554,2022-12-10,2022-12-30,False,i16,POINT (28.70048 45.57835)
9483,0.1675,0.1030,0.043,0.0678,0.0460,0.10960,0.23980,0.26430,0.2832,0.2783,0.720535,NaT,46.693510,35.051003,p555,2022-12-10,2022-12-30,False,i16,POINT (35.05100 46.69351)
9484,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,2022-07-30,50.451929,33.668204,p556,2022-12-10,2022-12-30,False,i16,POINT (33.66820 50.45193)


In [10]:
# cpied from learning_about-data.ipynb
df = DF.copy()
df = df[(df.NDVI) != 0] # drop invalid points
VEG_INDICES_NAMES = veg_indices.add_veg_indices(df) + ['NDVI'] 
VEG_DIFF_NAMES = veg_indices.add_veg_diff(df, VEG_INDICES_NAMES)
NUMERIC_COLS = BANDS + VEG_INDICES_NAMES + VEG_DIFF_NAMES
df = df.dropna(subset=['sample_idx']) # removes the first image, because sample_idx is NaN there


""" 
# only look at samples 6, 7, 8, 9
new_df = None
for i in range(6, 9 + 1):

    curr_df = df[df["sample_idx"] == f"s{i}"]
    if(type(new_df) == type(None)):
        new_df = curr_df
    else:
        new_df = pd.concat([new_df, curr_df])
df = new_df
"""

print(df.columns, df.shape)


# For each 3-week image, standarize each column
df = utilities.get_rm_outlier_standarize(df, NUMERIC_COLS, rm_outliers=False)




Added:  ['DVI', 'RVI', 'ARVI', 'PSSRa', 'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv', 'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI']
Added:  ['sample_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff', 'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff', 'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff', 'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff', 'NDWI_diff', 'NDSI_diff', 'NDVI_diff']
Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
       'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
       'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'PSSRa',
       'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv',
       'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI',
       'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_d

### Upsample harvevnt == true

In [13]:
har_evnt_true_df = df[df["har_evnt"]]
har_evnt_false_df = df[df["har_evnt"] == False]
print(har_evnt_true_df.shape)
print(har_evnt_false_df.shape)

(328, 66)
(6350, 66)


In [17]:
from sklearn.utils import resample

upsampled_har_evnt_true_df = resample(har_evnt_true_df, n_samples=har_evnt_false_df.shape[0],
                                                 replace=True, random_state=42)
upsampled_har_evnt_true_df.shape

(6350, 66)

In [19]:
df = pd.concat([upsampled_har_evnt_true_df, har_evnt_false_df])
df.shape

(12700, 66)

In [20]:
def get_one_hot(df:pd.DataFrame, colName: str, retArrays=False):
    lookupTable, indexed_dataSet = np.unique(df[colName], return_inverse=True)
    tpls = tf.keras.utils.to_categorical(indexed_dataSet)

    arrays = tpls.T[1:] # remove one column (not needed)
    arrays = arrays.astype(np.float32) # tensorflow does not support bools, so float 32 is the best we can do :/
    if(retArrays):
        return arrays

    return arrays.T # return to n by m (n tuples of size m)





In [21]:
df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NDTI_diff,NDMI_diff,MSI_diff,GCI_diff,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx
4839,1.385954,1.774244,0.129085,0.163238,0.334403,0.201913,-0.340219,-0.490786,-0.587137,-0.512474,...,0.286452,-1.452412,2.032427,-1.192398,0.591848,1.646797,-0.517996,-1.819393,-0.534278,s7
5507,1.060614,0.635746,0.440772,0.356287,0.354920,0.597320,0.689155,0.765114,0.516060,0.858730,...,-0.054707,-0.758196,0.209549,-0.362585,0.447302,0.036857,0.310800,-0.319903,-0.133213,s8
4848,-0.187346,-0.245464,-0.103838,-0.059468,0.278830,0.111869,-0.316513,-0.288947,-0.212522,-0.256863,...,0.761207,-0.109927,-0.069356,-0.228613,0.368772,0.118269,-0.266088,0.002666,-0.603419,s7
4728,-0.153022,-0.029013,-0.250125,-0.332513,-0.246332,-0.474782,-0.943350,-1.027254,-1.050201,-1.103574,...,0.701232,-1.304036,1.226621,-0.758460,0.446447,1.022054,-0.664009,-1.163051,-0.292966,s7
5196,-0.643317,-0.868245,-0.232785,-0.251894,-0.187857,-0.274461,-0.413436,-0.359850,-0.393159,-0.371695,...,1.163120,-0.398016,-0.345067,-0.013404,0.395945,-0.380607,0.075882,0.267730,-0.348433,s8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9475,3.322104,2.328019,0.132010,0.231196,0.095625,0.472431,1.725177,2.015223,2.066392,2.180961,...,-4.826355,0.732779,-1.169280,1.612652,-0.915200,-1.344209,0.282752,0.795201,0.566191,s15
9478,-1.628924,-1.062433,3.578395,3.457746,3.404489,3.259756,3.083942,2.963168,3.031307,2.897218,...,-0.668641,2.848363,-2.258848,-0.110247,1.915134,-2.202332,2.572595,2.716314,-2.818478,s15
9481,1.843392,1.368079,-0.275171,-0.257938,-0.181384,-0.168046,-0.179112,-0.158634,-0.057645,-0.060312,...,0.114312,-0.434865,0.208224,0.134505,-0.290227,0.266190,-0.606048,-0.316902,0.445377,s15
9482,-0.009429,-0.141409,-0.468817,-0.419537,-0.416868,-0.447121,-0.460629,-0.493758,-0.450153,-0.492917,...,0.411511,-0.449486,0.316610,0.088229,-0.345191,0.438725,-0.761268,-0.373365,0.573503,s15


In [22]:
df_X = df[VEG_DIFF_NAMES]
df_X = df_X.drop(['DVI_diff'], axis=1) # drop nana

In [23]:
X = df_X.to_numpy().reshape((df_X.shape[0], df_X.shape[1], 1))

In [24]:
Y = df["har_evnt"].to_numpy().astype(np.float32)
#df_Y = df_Y.reshape((df_Y.shape[0], 1))

In [25]:
X.shape

(12700, 21, 1)

In [26]:
X

array([[[ 1.16120678],
        [ 0.5186668 ],
        [-0.66685818],
        ...,
        [-0.51799586],
        [-1.81939281],
        [-0.53427836]],

       [[-0.02536285],
        [-1.07951348],
        [ 0.09868929],
        ...,
        [ 0.31080032],
        [-0.31990275],
        [-0.13321314]],

       [[ 0.59012555],
        [ 1.21949278],
        [-0.62796458],
        ...,
        [-0.26608761],
        [ 0.00266588],
        [-0.60341929]],

       ...,

       [[-0.36984042],
        [ 0.48810307],
        [-0.29588152],
        ...,
        [-0.60604802],
        [-0.31690166],
        [ 0.44537728]],

       [[-0.43992693],
        [ 0.97390859],
        [-0.26861977],
        ...,
        [-0.76126849],
        [-0.37336542],
        [ 0.57350275]],

       [[-2.16175383],
        [ 0.38730427],
        [ 2.64524903],
        ...,
        [ 0.10413741],
        [ 0.64074903],
        [ 0.96223682]]])

In [28]:

def convert_to_1hot(categorical_col_name:str):
    names = []
    arrays = get_one_hot(df, categorical_col_name, retArrays=True)
    for i in range(len(arrays)):
        arr = arrays[i]
        name = f'{categorical_col_name}_1hot_{i}'
        df[name] = arr
        names.append(name)
    return names
    """
one_hot_names_X = convert_to_1hot("sample_idx")
one_hot_names_Y = convert_to_1hot("har_evnt")

print(df.columns)
print(one_hot_names_X)
print(one_hot_names_Y)
    """

In [29]:
X.shape

(12700, 21, 1)

In [30]:
X.shape[1], X.shape[2]

(21, 1)

In [31]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

model = tf.keras.Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X.shape[1],X.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    BatchNormalization(),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=8, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

#model.compile(optimizer='adam', loss='mse', metrics=['mae'])

2023-05-05 15:04:16.952994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 19, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 32)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 288)               0         
                                                                 
 batch_normalization (BatchN  (None, 288)              1152      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                18496     
                                                                 
 dense_1 (Dense)             (None, 32)                2

In [33]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])


In [34]:
model.fit(X, Y, epochs=30, validation_split=0.2)


Epoch 1/30
318/318 [==============================] - 4s 5ms/step - loss: 0.4407 - binary_accuracy: 0.8060 - val_loss: 0.5979 - val_binary_accuracy: 0.6543
Epoch 2/30
318/318 [==============================] - 1s 3ms/step - loss: 0.3223 - binary_accuracy: 0.8774 - val_loss: 1.1588 - val_binary_accuracy: 0.4197
Epoch 3/30
318/318 [==============================] - 2s 5ms/step - loss: 0.2791 - binary_accuracy: 0.9004 - val_loss: 1.5026 - val_binary_accuracy: 0.3476
Epoch 4/30
318/318 [==============================] - 1s 5ms/step - loss: 0.2601 - binary_accuracy: 0.9025 - val_loss: 1.4092 - val_binary_accuracy: 0.4205
Epoch 5/30
318/318 [==============================] - 1s 3ms/step - loss: 0.2410 - binary_accuracy: 0.9146 - val_loss: 1.1710 - val_binary_accuracy: 0.4535
Epoch 6/30
318/318 [==============================] - 1s 3ms/step - loss: 0.2253 - binary_accuracy: 0.9196 - val_loss: 1.0650 - val_binary_accuracy: 0.5185
Epoch 7/30
318/318 [==============================] - 1s 4ms/ste

In [216]:
a = tf.constant(X)
tf.math.l2_normalize(a).numpy()

array([[[ 0.        ,  0.        ,  0.        , ...,  0.00552828,
         -0.00659141, -0.00677609],
        [ 0.00757559,  0.        ,  0.        , ..., -0.00238965,
         -0.01646883, -0.01451381],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00472415,
         -0.00212759, -0.00230309],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.00232412,
          0.00348978,  0.00299009]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.00921513,
          0.00537546,  0.00040839],
        [ 0.00757559,  0.        ,  0.        , ...,  0.0002729 ,
          0.00560671,  0.00508392],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00203002,
          0.00129858,  0.00159386],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.01397219,
         -0.00468563, -0.00910651]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.01151   ,
         -0.00425536, -0.00547394],
        [ 0.00757559,  0.        ,  0.        , ...,  0.00533579,
         -0.

In [75]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy =  train_test_split(X, Y, test_size = 0.4,random_state=42) 

evaluate_model(trainX, trainy, testX, testy)

ValueError: in user code:

    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/tahaalnufaili/

In [ ]:
testX.shape, trainy.shape

((1249, 1), (834, 57))